In this notebook, an end-to-end pipeline is developed for the Click-Through Rate prediction task on Kaggle (https://www.kaggle.com/c/avazu-ctr-prediction). The goal is to predict the probability of whether the 'click' variable is zero or one for each 'id'. The output of the pipeline is a csv file with two columns, one with the id's and one with the predicted click probabilities. 

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('train.csv', nrows=1000000) # due to limited memory, the whole file cannot be used
pd.options.display.max_columns = 30
data.isnull().values.any()

False

The data seems to be cleaned of missing values.

In [2]:
data.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,ddd2926e,44956a24,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,96809ac8,711ee120,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,b3cf8def,8a4875bd,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,e8275b8f,6332421a,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,07d7df22,a99f214a,9644d0bf,779d90c2,1,0,18993,320,50,2161,0,35,-1,157


Having a look at the first few rows of the data is useful to come up with further enquiries. First, we want to know what the dimensions of the data are (e.g. cross-sectional, time series, panel data). Therefore we want to know how many rows of data we have per ad id. 

In [3]:
len(data['id'].unique())

1000000

This means that we have cross-sectional data (one observation per ad id). However, there is a time dimension in the data represented by the column 'hour', so let's see how many different time points we have.

In [4]:
len(data['hour'].unique())

6

Since we have not been able to load the full dataset, we do not have time-points for all 10 days. Moreover, even if we had this data, because we have only one observation per ad, the dynamics are not of fundamental importance, hence for simplicity we drop the 'hour' variable and focus only on the train.csv dataset (the test.csv was included in the Kaggle competition separately mainly because it provided additional time points).

Now we are in a position to transform our dataset into a tidy dataset, meaning that each column is a variable and each row an observation, hence 'id' should be an index. In addition, it is instrumental to take a closer look at the data as well.

In [5]:
data = data.drop(columns=['hour'])
data = data.set_index(['id'])
data.describe(include='all')

,click,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
count,1000000.000000,1000000.000000,1000000.000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
unique,NaN,NaN,NaN,2075,2030,21,2309,156,23,83431,313002,4581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,85f751fd,c4e18dd6,50e219e0,ecad2386,7801e8d9,07d7df22,a99f214a,6b9769f2,8a4875bd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,332893,348412,360056,667107,707429,679869,840265,5886,59771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,0.160219,1005.088166,0.229922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.025540,0.223363,18262.203151,318.965808,56.495546,2041.031112,1.452262,190.779388,45505.857239,69.936118
std,0.366809,1.156928,0.464627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.453899,0.667164,3510.366302,19.452897,36.546944,441.200965,1.362637,273.439286,49843.810147,38.513846
min,0.000000,1001.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,375.000000,120.000000,20.000000,112.000000,0.000000,33.000000,-1.000000,13.000000
25%,0.000000,1005.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,15707.000000,320.000000,50.000000,1722.000000,0.000000,35.000000,-1.000000,43.000000
50%,0.000000,1005.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,19251.000000,320.000000,50.000000,2161.000000,1.000000,39.000000,-1.000000,61.000000
75%,0.000000,1005.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,21153.000000,320.000000,50.000000,2420.000000,3.000000,297.000000,100084.000000,79.000000


Note that the average click frequency over the entire dataset is 0.16, which is significantly less than half. According to the summary, some of the variables that were described on Kaggle as categorical are not coded as such (they are probably integer): 'C1', 'device_type', 'device_conn_type' and 'C14' - 'C21'. Instead of transforming them to categorical variables, we look at the number of unique values and datatype, which are most important. (Note that the only non-categorical variable is 'banner_pos', which takes a few different values, so there do not seem to be outliers there.)

In [6]:
int_vars = ['C1', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']
for var in int_vars:
    print('number of levels: ', len(data[var].unique()))
    print('data type: ', data[var].dtype)
    print('')

number of levels:  7
data type:  int64

number of levels:  4
data type:  int64

number of levels:  4
data type:  int64

number of levels:  606
data type:  int64

number of levels:  8
data type:  int64

number of levels:  9
data type:  int64

number of levels:  162
data type:  int64

number of levels:  4
data type:  int64

number of levels:  41
data type:  int64

number of levels:  161
data type:  int64

number of levels:  35
data type:  int64



We can see that the following variables have many levels (ad hod defined as > 10): 'device_id', 'device_ip', 'device_model', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'C14', 'C17', 'C19', 'C20', 'C21'. In the case of 'device_ip' (essentially a user id) there are on average just three observations per level! To ensure that estimation remains robust and tractable (and to make sure that after creating binary dummies, the dataframe still fits into memory), it would be useful to perform a Principal Component Analysis on the collection of binary dummies from each variable to reduce the number of levels per feature by projecting the high dimensional feature space onto a more manageable low dimensional space with, say, 5 features. For simplicity, however, we will just drop the categorical features that have many levels. 


In [7]:
data = data.drop(columns=['device_id', 'device_ip', 'device_model', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'C14', 'C17', 'C19', 'C20', 'C21'])

Before proceeding, we wish to explore the relationship between the target variable 'id' and each of the features. Even though 'banner_pos' is non-categorical, it takes only a small number of integer values.

In [8]:
features  = ['C1', 'banner_pos', 'device_type', 'device_conn_type', 'C15', 'C16', 'C18']
for feature in features:
    print(data[['click', feature]].groupby(feature).mean())


         click
C1            
1001  0.074766
1002  0.228194
1005  0.161499
1007  0.043275
1008  0.087652
1010  0.075076
1012  0.032393
               click
banner_pos          
0           0.151534
1           0.191514
2           0.139881
3           0.000000
4           0.145833
5           0.087652
7           0.060000
                click
device_type          
0            0.228194
1            0.159642
4            0.074682
5            0.080309
                     click
device_conn_type          
0                 0.165780
2                 0.117138
3                 0.082305
5                 0.036667
         click
C15           
120   0.000000
216   0.177866
300   0.404485
320   0.151062
480   0.500000
728   0.052057
768   0.555556
1024  0.875000
         click
C16           
20    0.000000
36    0.177866
50    0.151356
90    0.052057
250   0.430206
320   0.500000
480   0.038685
768   0.875000
1024  0.555556
        click
C18          
0    0.174869
1    0.061379
2    0.3424

The levels of each of these features differ greatly in their impact on the target variable, hence all are potentially useful features. 'banner_pos' seems to have an overall decreasing but somewhat non-monotonous effect on 'click', hence in future work higher order terms could be added to the regression equation. In order to use the sklearn library, we have to create dummy variables for each level for each categorical variable. (The dummy variable for the first level has to be dropped as there will be multicollinearity otherwise.) Given that not a lot of memory is available, it is not possible to create dummies for all categorical features that have many levels. 

In [9]:
cols_to_transform = ['C1', 'device_type', 'device_conn_type', 'C15', 'C16', 'C18']
data = pd.get_dummies(data, columns = cols_to_transform, drop_first=True)
data.head()

,click,banner_pos,C1_1002,C1_1005,C1_1007,C1_1008,C1_1010,C1_1012,device_type_1,device_type_4,device_type_5,device_conn_type_2,device_conn_type_3,device_conn_type_5,C15_216,...,C15_480,C15_728,C15_768,C15_1024,C16_36,C16_50,C16_90,C16_250,C16_320,C16_480,C16_768,C16_1024,C18_1,C18_2,C18_3
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.000009e+18,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1.000017e+19,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1.000037e+19,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1.000064e+19,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1.000068e+19,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


Now we can start with training a model. To keep things simple our classifier will be a logistic regression model. This model is easy to interpret and has a short training time. We will train the model on a training set and then validate it on a test set. The sklearn library only works with scaled non-categorical variables, hence we have to standardize 'banner_pos'. Standardization is performed on the training set, after which the same transformation is performed on the test set and total set for the final output. To evaluate the model, the average log loss is computed, as per task requirement, for both the train set and test set.

In [10]:
# from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

X = data[data.columns[1:]].values.astype(float)
y = data[data.columns[0]].values.astype(float)
index = data.index

del data # save memory

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=44) # 80 / 20 split

scaler = StandardScaler().fit(X_train[:, [0]]) # first column corresponds to 'banner_pos'

X_train[:, [0]] = scaler.transform(X_train[:, [0]])
X_test[:, [0]] = scaler.transform(X_test[:, [0]])
X[:, [0]] = scaler.transform(X[:, [0]])

logistic_regression = LogisticRegression(C=1000000) # approximately no regularization
logistic_regression.fit(X_train, y_train)
y_pred_train = logistic_regression.predict_proba(X_train)[:, 1]
y_pred_test = logistic_regression.predict_proba(X_test)[:, 1]

train_error = log_loss(y_train, y_pred_train, 1e-15)
test_error = log_loss(y_test, y_pred_test, 1e-15)

print('train_error: ', train_error)
print('test_error: ', test_error)

train_error:  0.41723291116037425
test_error:  0.4174995777223711


First, notice that the train and test error are close in magnitude, which means that even without regularization, there appears to be no overfitting. Alternatively, the test data is very similar to the training data. It is therefore unnecessary to do hyper parameter tuning, because regularization is unlikely to improve the test error. Secondly, it would be good to put the test error (which is essentially a score for the model), into context. The most naive prediction method (e.g. without knowing anything about the data), would be to set all probabilities to 0.5. A somwehat less naive method would be to say that all probabilities are equal to the click frequency(0.16 in this case), disregarding the information in the features.

In [11]:
print('average log loss naive method: ', log_loss(y_train, [0.50]*len(y_train), 1e-15))
print('average log loss less naive method: ', log_loss(y_train, [0.16]*len(y_train), 1e-15))

average log loss naive method:  0.6931471805599458
average log loss less naive method:  0.44000359780175946


The log loss of our model is lower than both naive methods. However, the model could be significantly improved if more data could is used (more rows and more features). This is partly because not all levels are available for most features due to the limited number of rows (covering only 6 hours out of 10 days, or 2.5%) and partly because few features were used. Presumably this would also allow for non-trivial tuning of the hyper-parameters. These limitations are manifested in a small number of distinct predictions for both training and test data.

In [12]:
print('distinct predictions training set: ', len(np.unique(y_pred_train)))
print('distinct predictions test set: ', len(np.unique(y_pred_test)))

distinct predictions for training set:  152
distinct predictions for test set:  132


The final step in the pipeline is to write all predictions to a csv file.

In [13]:
y_pred = logistic_regression.predict_proba(X)[:, 1]
df = pd.DataFrame(y_pred, index=index)
df.to_csv('output_predictions.csv')